In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import boto3

s3 = boto3.client('s3')
bucket_name = 'nexttrendco'

# Download Walmart files from S3 
def download_from_s3(bucket_name, s3_key, local_filename):
    s3.download_file(bucket_name, s3_key, local_filename)
    print(f"Downloaded {s3_key} to {local_filename}")

# download training, validation, and test datasets
download_from_s3(bucket_name, 'walmart/train.csv', 'train.csv')
download_from_s3(bucket_name, 'walmart/val.csv', 'val.csv')
download_from_s3(bucket_name, 'walmart/test.csv', 'test.csv')

# load data
train_walmart = pd.read_csv('train.csv')
val_walmart = pd.read_csv('val.csv')
test_walmart = pd.read_csv('test.csv')

# print first few rows 
print("Training Data Sample:")
print(train_walmart.head())  

print("\nValidation Data Sample:")
print(val_walmart.head()) 

print("\nTest Data Sample:")
print(test_walmart.head()) 

Downloaded walmart/train.csv to train.csv
Downloaded walmart/val.csv to val.csv
Downloaded walmart/test.csv to test.csv
Training Data Sample:
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False

Validation Data Sample:
   Store  Dept        Date  Weekly_Sales  IsHoliday     lag_1     lag_7  \
0      1     1  2010-03-12      21043.39      False  21827.90       NaN   
1      1     1  2010-05-07      17413.94      False  16555.11  22136.64   
2      1     1  2010-05-21      14773.04      False  18926.74  57258.43   
3      1     1  2010-06-11      16637.62      False  17558.09  16145.35   
4      1     1  2010-06-18      16216.27      False  16637.62  16555.11   

   rolling_mean_7  holiday_sales_interaction  day_of_week  month  
0  

In [31]:
# preprocess and fill missing values
train_walmart.ffill(inplace=True)
val_walmart.ffill(inplace=True)
test_walmart.ffill(inplace=True)

In [44]:
# Feature Engineering for Training Set
train_walmart['lag_1'] = train_walmart['Weekly_Sales'].shift(1)
train_walmart['lag_7'] = train_walmart['Weekly_Sales'].shift(7)
train_walmart['rolling_mean_7'] = train_walmart['Weekly_Sales'].rolling(window=7).mean()

# Correct column names (use IsHoliday_True)
train_walmart['promo_holiday_interaction'] = train_walmart['promo_holiday_interaction'] * train_walmart['IsHoliday_True']

# Interaction feature using 'IsHoliday_True'
train_walmart['holiday_interaction'] = train_walmart['IsHoliday_True'] * train_walmart['Weekly_Sales']

# Print 
print("Train Walmart DataFrame with Features:")
print(train_walmart.head())

Train Walmart DataFrame with Features:
   Store  Dept        Date  Weekly_Sales     lag_1  lag_7  rolling_mean_7  \
0      1     1  2010-02-05      24924.50       NaN    NaN             NaN   
1      1     1  2010-02-12      46039.49  24924.50    NaN             NaN   
2      1     1  2010-02-19      41595.55  46039.49    NaN             NaN   
3      1     1  2010-02-26      19403.54  41595.55    NaN             NaN   
4      1     1  2010-03-05      21827.90  19403.54    NaN             NaN   

   promo_holiday_interaction  IsHoliday_True  holiday_interaction  
0                       0.00           False                 0.00  
1                   46039.49            True             46039.49  
2                       0.00           False                 0.00  
3                       0.00           False                 0.00  
4                       0.00           False                 0.00  


In [46]:
# Feature engineering for Validation Set
val_walmart['lag_1'] = val_walmart['Weekly_Sales'].shift(1)
val_walmart['lag_7'] = val_walmart['Weekly_Sales'].shift(7)
val_walmart['rolling_mean_7'] = val_walmart['Weekly_Sales'].rolling(window=7).mean()

# Correcting the column name to 'IsHoliday_True'
val_walmart['promo_holiday_interaction'] = val_walmart['promo_holiday_interaction'] * val_walmart['IsHoliday_True']

# Interaction feature using 'IsHoliday_True' 
val_walmart['holiday_interaction'] = val_walmart['IsHoliday_True'] * val_walmart['Weekly_Sales']

# Print 
print("Validation Walmart DataFrame with Features:")
print(val_walmart.head())


Validation Walmart DataFrame with Features:
   Store  Dept        Date  Weekly_Sales     lag_1  lag_7  rolling_mean_7  \
0      1     1  2010-03-12      21043.39       NaN    NaN             NaN   
1      1     1  2010-05-07      17413.94  21043.39    NaN             NaN   
2      1     1  2010-05-21      14773.04  17413.94    NaN             NaN   
3      1     1  2010-06-11      16637.62  14773.04    NaN             NaN   
4      1     1  2010-06-18      16216.27  16637.62    NaN             NaN   

   promo_holiday_interaction  day_of_week  month  holiday_interaction  \
0                        0.0            4      3                  0.0   
1                        0.0            4      5                  0.0   
2                        0.0            4      5                  0.0   
3                        0.0            4      6                  0.0   
4                        0.0            4      6                  0.0   

   IsHoliday_True  
0           False  
1           Fa

In [49]:
# Feature engineering for test dataset tc.
test_walmart['lag_1'] = test_walmart['lag_1'].shift(1)
test_walmart['lag_7'] = test_walmart['lag_7'].shift(7)  
test_walmart['rolling_mean_7'] = test_walmart['rolling_mean_7'].rolling(window=7).mean()

# Create interaction features without using Weekly_Sales
test_walmart['holiday_interaction'] = test_walmart['IsHoliday_True'] * test_walmart['IsHoliday_True']

# Print
print("Test Walmart DataFrame with Features:")
print(test_walmart.head())

Test Walmart DataFrame with Features:
   Store  Dept        Date  lag_1 lag_7  rolling_mean_7  \
0      1     1  2012-11-02   None  None             NaN   
1      1     1  2012-11-09    NaN  None             NaN   
2      1     1  2012-11-16  False  None             NaN   
3      1     1  2012-11-23  False  None             NaN   
4      1     1  2012-11-30  False  None             NaN   

   promo_holiday_interaction  IsHoliday_True  holiday_interaction  
0                      False           False                False  
1                      False           False                False  
2                      False           False                False  
3                       True            True                 True  
4                      False           False                False  


In [51]:
# Min-Max Scaling for the selected numerical features excluding 'Weekly_Sales' the test dataset
scaler = MinMaxScaler()

# List of columns to scale, excluding 'Weekly_Sales' in test dataset
scaled_columns = ['lag_1', 'lag_7', 'rolling_mean_7', 'promo_holiday_interaction']

# Apply scaling to the training and validation datasets
train_walmart[scaled_columns] = scaler.fit_transform(train_walmart[scaled_columns])
val_walmart[scaled_columns] = scaler.transform(val_walmart[scaled_columns])

# For test dskip scaling on 'Weekly_Sales' and only scale available columns
test_walmart[scaled_columns] = scaler.transform(test_walmart[scaled_columns])

# Print
print("Test Walmart DataFrame after scaling:")
print(test_walmart.head())

Test Walmart DataFrame after scaling:
   Store  Dept        Date  lag_1  lag_7  rolling_mean_7  \
0      1     1  2012-11-02    NaN    NaN             NaN   
1      1     1  2012-11-09    NaN    NaN             NaN   
2      1     1  2012-11-16    0.0    NaN             NaN   
3      1     1  2012-11-23    0.0    NaN             NaN   
4      1     1  2012-11-30    0.0    NaN             NaN   

   promo_holiday_interaction  IsHoliday_True  holiday_interaction  
0                        0.0           False                False  
1                        0.0           False                False  
2                        0.0           False                False  
3                        1.0            True                 True  
4                        0.0           False                False  


In [52]:
# One-Hot Encoding on categorical columns
train_walmart = pd.get_dummies(train_walmart, columns=['Store', 'Dept'], drop_first=True)
val_walmart = pd.get_dummies(val_walmart, columns=['Store', 'Dept'], drop_first=True)
test_walmart = pd.get_dummies(test_walmart, columns=['Store', 'Dept'], drop_first=True)

# Print
print("Train Walmart DataFrame with One-Hot Encoding:")
print(train_walmart.head())

print("Validation Walmart DataFrame with One-Hot Encoding:")
print(val_walmart.head())

print("Test Walmart DataFrame with One-Hot Encoding:")
print(test_walmart.head())

Train Walmart DataFrame with One-Hot Encoding:
         Date  Weekly_Sales     lag_1  lag_7  rolling_mean_7  \
0  2010-02-05      0.042851       NaN    NaN             NaN   
1  2010-02-12      0.073097  0.042851    NaN             NaN   
2  2010-02-19      0.066732  0.073097    NaN             NaN   
3  2010-02-26      0.034942  0.066732    NaN             NaN   
4  2010-03-05      0.038415  0.034942    NaN             NaN   

   promo_holiday_interaction  IsHoliday_True  holiday_interaction  Store_2  \
0                   0.001150           False                 0.00    False   
1                   0.067499            True             46039.49    False   
2                   0.001150           False                 0.00    False   
3                   0.001150           False                 0.00    False   
4                   0.001150           False                 0.00    False   

   Store_3  ...  Dept_90  Dept_91  Dept_92  Dept_93  Dept_94  Dept_95  \
0    False  ...    False  

In [53]:
# Ensure the 'Date' column is in datetime format
train_walmart['Date'] = pd.to_datetime(train_walmart['Date'])
val_walmart['Date'] = pd.to_datetime(val_walmart['Date'])
test_walmart['Date'] = pd.to_datetime(test_walmart['Date'])

# Sort the data chronologically by the 'Date' column
train_walmart = train_walmart.sort_values('Date')
val_walmart = val_walmart.sort_values('Date')
test_walmart = test_walmart.sort_values('Date')

# Display first few rows
print("Train Walmart Data (chronologically sorted):")
print(train_walmart.head())

Train Walmart Data (chronologically sorted):
             Date  Weekly_Sales     lag_1     lag_7  rolling_mean_7  \
0      2010-02-05      0.042851       NaN       NaN             NaN   
277665 2010-02-05      0.029425  0.030801  0.030097        0.055201   
277808 2010-02-05      0.011731  0.021941  0.017781        0.037953   
277951 2010-02-05      0.022646  0.008669  0.009342        0.011676   
278094 2010-02-05      0.035876  0.023277  0.021539        0.051509   

        promo_holiday_interaction  IsHoliday_True  holiday_interaction  \
0                         0.00115           False                  0.0   
277665                    0.00115           False                  0.0   
277808                    0.00115           False                  0.0   
277951                    0.00115           False                  0.0   
278094                    0.00115           False                  0.0   

        Store_2  Store_3  ...  Dept_90  Dept_91  Dept_92  Dept_93  Dept_94  \
0    

In [54]:
# Data Splitting 
train_ratio = 0.8  # 80% for training
val_ratio = 0.15   # 15% for validation
test_ratio = 0.05  # 5% for testing

# split points for train, validation, and test data
train_end = int(len(train_walmart) * train_ratio)
val_end = int(len(train_walmart) * (train_ratio + val_ratio))

# Split  data
train_walmart = train_walmart[:train_end]
val_walmart = train_walmart[train_end:val_end]
test_walmart = train_walmart[val_end:]

# Verify splits
print("Train Walmart Data Shape:", train_walmart.shape)
print("Validation Walmart Data Shape:", val_walmart.shape)
print("Test Walmart Data Shape:", test_walmart.shape)

Train Walmart Data Shape: (337256, 132)
Validation Walmart Data Shape: (0, 132)
Test Walmart Data Shape: (0, 132)


In [55]:
# Save the preprocessed data as CSVs
train_walmart.to_csv('train_preprocessed.csv', index=False)
val_walmart.to_csv('val_preprocessed.csv', index=False)
test_walmart.to_csv('test_preprocessed.csv', index=False)

In [ ]:
# S3 paths
s3_train_path = 's3://nexttrendco/walmart/train_preprocessed.csv'
s3_val_path = 's3://nexttrendco/walmart/val_preprocessed.csv'
s3_test_path = 's3://nexttrendco/walmart/test_preprocessed.csv'

# Upload to S3
s3.upload_file('train_preprocessed.csv', bucket_name, 'walmart/train_preprocessed.csv')
s3.upload_file('val_preprocessed.csv', bucket_name, 'walmart/val_preprocessed.csv')
s3.upload_file('test_preprocessed.csv', bucket_name, 'walmart/test_preprocessed.csv')

# Confirm the upload
print("Data uploaded successfully")